In [18]:
import json
import pyodbc

In [19]:
class LayoutDB:
    def __init__(self):
        self.server_solmicro = r'SERVIDOR\SOLMICRO6'
        self.database_solmicro = 'SolmicroERP6_PruebasSub'
        self.username_solmicro = 'sa'
        self.password_solmicro = 'Altai2021'
        self.connection_string_solmicro = f'DRIVER={{SQL Server}};SERVER={self.server_solmicro};DATABASE={self.database_solmicro};UID={self.username_solmicro};PWD={self.password_solmicro}'
        self.connection = None
        self.tables_info = []

    def Open_Conn_Solmicro(self):
        try:
            self.connection = pyodbc.connect(self.connection_string_solmicro)
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def __del__(self):
        self.CloseConnection()

    def CloseConnection(self):
        try:
            self.connection = pyodbc.connect(self.connection_string_solmicro)
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def export_tables(self):
        cursor = self.connection.cursor()
        # Obtener todas las tablas en la base de datos
        tables_query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'"
        tables = cursor.execute(tables_query).fetchall()
        for table in tables:
            table_name = table[0]
            table_info = {
                'tableName': table_name,
                'fields': [],
                'relatedTables': []
            }
            # Obtener información sobre las columnas de la tabla
            columns_query = f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='{table_name}'"
            # Obtener las columnas que son llaves foráneas en esta tabla
            # foreign_keys_query = f"""SELECT t.name AS TableName,c.name AS ColumnName,rt.name AS RelatedTableName
            #     FROM sys.foreign_key_columns AS fkc INNER JOIN sys.tables AS t ON t.object_id = fkc.parent_object_id 
            #     INNER JOIN sys.columns AS c ON c.column_id = fkc.parent_column_id AND c.object_id = t.object_id
            #     INNER JOIN sys.tables AS rt ON rt.object_id = fkc.referenced_object_id WHERE t.name = '{table_name}'"""
            foreign_keys_query = f"""SELECT OBJECT_NAME(fk.parent_object_id) AS TableName, c.name AS ColumnName, OBJECT_NAME(fk.referenced_object_id) AS RelatedTableName
            FROM sys.foreign_keys AS fk
            INNER JOIN sys.foreign_key_columns AS fkc ON fk.object_id = fkc.constraint_object_id
            INNER JOIN sys.columns AS c ON fkc.parent_column_id = c.column_id AND fkc.parent_object_id = c.object_id
            WHERE OBJECT_NAME(fkc.referenced_object_id) = '{table_name}'"""
            columns = cursor.execute(columns_query).fetchall()
            foreign_keys = cursor.execute(foreign_keys_query).fetchall()
            for column in columns:
                column_name, column_type = column
                field_info = {
                    'nombre': column_name,
                    'tipo': column_type
                }
                table_info['fields'].append(field_info)
            for foreign_key in foreign_keys:
                # related_table_name = foreign_key[2]
                related_table_name = foreign_key[0]
                foreign_key_column = foreign_key[1]
                if related_table_name not in table_info['relatedTables']:
                    table_info['relatedTables'].append({
                        "relatedTable": related_table_name,
                        "foreignKey": foreign_key_column
                        })
                    
            self.tables_info.append(table_info)
        cursor.close()
        self.connection.close()
        # return self.tables_info

    def Create_Estructura(self):
        final_structure = {
            'role': 'system',
            'content': json.dumps(self.tables_info, separators=(',', ':'))
        }
        # Guardar la estructura en un archivo de texto
        with open('estructura_tablas.txt', 'w') as file:
            file.write(str(final_structure))

    def get_table_relationships(self):
        cursor = self.connection.cursor()
        for table_info in self.tables_info:
            table_name = table_info['tableName']
            table_info['relatedTables'] = []
        # Obtener las columnas que son llaves foráneas en esta tabla
            foreign_keys_query = f"""SELECT t.name AS TableName,c.name AS ColumnName,rt.name AS RelatedTableName
                FROM sys.foreign_key_columns AS fkc INNER JOIN sys.tables AS t ON t.object_id = fkc.parent_object_id 
                INNER JOIN sys.columns AS c ON c.column_id = fkc.parent_column_id AND c.object_id = t.object_id
                INNER JOIN sys.tables AS rt ON rt.object_id = fkc.referenced_object_id WHERE t.name = '{table_name}'"""
            foreign_keys = cursor.execute(foreign_keys_query).fetchall()

            for foreign_key in foreign_keys:
                related_table_name = foreign_key[2]
                if related_table_name not in table_info['relatedTables']:
                    table_info['relatedTables'].append(related_table_name)

        cursor.close()

In [21]:
instace_layoutDB = LayoutDB()
instace_layoutDB.Open_Conn_Solmicro()
instace_layoutDB.export_tables()
instace_layoutDB.Create_Estructura()